### Workflow 

1. Use the labeled Dataset class that you wrote for milestone 2 and separate the images in the labeled folder into **train** and **validation** sets (having a validation set is especially important when training on a limited number of samples, since the danger of overfitting to training data is high).
1. Construct and train your supervised melanoma classifier. How does it perform on the test set?
1. Transfer learning is a popular machine learning (ML) approach when training data is scarce. Either choose a pre-trained model out of the ones available in the `torchvision.models` package, or train your own on a (large) dataset of your choice, and fine-tune it on your melanoma data. How does its performance compare to the fully supervised classifier that you trained from scratch in step 2?
1. When transfer learning does not prove to be sufficient, another option that we have is to take advantage of the unlabeled data by employing a semi-supervised ML strategy. Semi-supervised learning is a rapidly developing field, so the best place to look for the state-of-the-art methods is the famous arXiv.org, an open-access archive.  containing thousands of articles across numerous scientific disciplines, including computer science. Learning to work with the arXiv is an important skill for ML engineers involved in solving non trivial problems. **Spend some time to find at least 2-3 different approaches that have shown to be successful in semi-supervised ML for computer vision applications, and list them** along with the arXiv references that you found. It's fine to search arxiv.org and just read abstracts. See
  * https://arxiv.org/search/?query=semi+supervised+image+classification&searchtype=all



### Resources

* https://livebook.manning.com/book/grokking-deep-learning-for-computer-vision/chapter-6
* https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
* https://livebook.manning.com/book/deep-learning-with-pytorch/chapter-2/81

In [6]:
url = 'https://liveproject-resources.s3.amazonaws.com/other/MelanomaDetection.zip'
pth = './data/MelanomaDetection.zip'
unp = './data/MelanomaDetection'

In [43]:
import torch
import torchvision
from sklearn.model_selection import train_test_split
from torch.nn import BCELoss, BCEWithLogitsLoss, CrossEntropyLoss #not sure which I'd like to use. 
from torch.nn import Linear
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader


In [8]:
%load_ext autoreload
%autoreload 2

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Create Training and Validation Sets

In [45]:
labeledSet = MILabeled(unp + '/labeled', ToTensor())

sd = 43
cs = len(labeledSet)
ci = range(0, cs)
train_idxs, test_idxs = train_test_split(ci, random_state=sd, train_size=0.75)

#todo: i think this wrong. this is no longer a Dataset object. 
# consider the following approaches 
# - https://github.com/taitsmp/liveproject_medical_images/blob/master/utils.py#L69
# - could you solve this with a sampler? https://pytorch.org/docs/stable/data.html#torch.utils.data.Sampler
train_ds = [labeledSet[i] for i in train_idxs]
test_ds = [labeledSet[i] for i in test_idxs]

### Construct the Model

In [27]:
mod = torchvision.models.resnet18(False, True)

##Todo how to change n

In [34]:
print(mod)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [32]:
mod.fc

Linear(in_features=512, out_features=1000, bias=True)

In [36]:
#swap out the fully connected layer.  
mod.fc = Linear(512, 2)

In [42]:

criterion = CrossEntropyLoss()
optimizer = Adam(mod.parameters(), lr=1e-3)
labeled_ds
dataloader = DataLoader()


### Write a Training Loop

In [ ]:

def train_epoch(epoch, mod, dataloader, criterion, optimizer):
    
    #set train to true
    
    #zero out the gradient. 
    
    #loop through batches in data loader
    for bi, batch in enumerate(dataloader):
        
    
    
    #forward pass
    
    #backward pass
    
    #calc and save the loss
    
    pass

def valid_epoch(epoch, mod, dataloader, criterion):
    
    #set train to false
    
    #loop through batches in data loader
    
    #forward pass
    
    #backward pass
    
    #calc and save the loss
    pass




In [ ]:

for i in range(num_epochs):
    
    train_epoch()

### Perform Transfer Learning